In [1]:
# ============================================================
# 0) Environment & callback fixes (disable W&B and Ray)
# ============================================================
import os
os.environ["WANDB_DISABLED"] = "true"
os.environ["WANDB_MODE"] = "disabled"
os.environ["TOKENIZERS_PARALLELISM"] = "false"

# Remove offending callbacks
!pip -q uninstall -y wandb >/dev/null
!pip -q uninstall -y ray >/dev/null

# Core libs
!pip -q install ultralytics==8.2.84 transformers==4.41.2 accelerate==0.33.0 timm==0.9.16 torchmetrics==1.4.0

import time, random, shutil, json
from pathlib import Path
import numpy as np, pandas as pd
from tqdm import tqdm
import torch
from ultralytics import YOLO

# Toggles
USE_VLM = True     # BLIP scene captions
USE_LLM = True     # FLAN-T5 inspection summaries
RUN_EVOLVE = False # quick hyperparam evolution (off by default)

SEED = 42
random.seed(SEED); np.random.seed(SEED); torch.manual_seed(SEED)

# ============================================================
# 1) Dataset paths (YOUR structure)
# ============================================================
ROOT = Path("/kaggle/input/yolo-annotated-wind-turbines-586x371")
DS   = ROOT / "NordTank586x371"
IMG_DIR_IN = DS / "images"
LBL_DIR_IN = DS / "labels"
assert IMG_DIR_IN.exists() and LBL_DIR_IN.exists(), "Dataset paths not found."

WORKDIR = Path("/kaggle/working")
PROJECT = "windturbine_yolo_llm_strong"
OUT = WORKDIR / PROJECT

for p in [
    OUT/"dataset/images/train", OUT/"dataset/images/val", OUT/"dataset/images/test",
    OUT/"dataset/labels/train", OUT/"dataset/labels/val", OUT/"dataset/labels/test",
    OUT/"artifacts"
]:
    p.mkdir(parents=True, exist_ok=True)

# ------------------------------------------------------------
# 1a) Index images + detect label status (without writing to input)
# ------------------------------------------------------------
images = sorted([*IMG_DIR_IN.glob("*.png"), *IMG_DIR_IN.glob("*.jpg"), *IMG_DIR_IN.glob("*.jpeg")])

def label_exists_and_nonempty(stem: str):
    lb = LBL_DIR_IN / f"{stem}.txt"
    if not lb.exists():
        return False, False
    try:
        txt = lb.read_text().strip()
        return True, (txt != "")
    except:
        return True, False

pairs = []
for im in images:
    exists, nonempty = label_exists_and_nonempty(im.stem)
    pairs.append({"img": im, "lbl_in": (LBL_DIR_IN/f"{im.stem}.txt") if exists else None, "has_obj": nonempty})

# Stratified split: positives vs backgrounds
pos = [p for p in pairs if p["has_obj"]]
neg = [p for p in pairs if not p["has_obj"]]

def split3(items, train_ratio=0.8, val_ratio=0.1, seed=SEED):
    items = items.copy()
    random.Random(seed).shuffle(items)
    n = len(items); n_train = int(n*train_ratio); n_val = int(n*val_ratio)
    return items[:n_train], items[n_train:n_train+n_val], items[n_train+n_val:]

tr_p, va_p, te_p = split3(pos)
tr_n, va_n, te_n = split3(neg)
train, val, test = tr_p + tr_n, va_p + va_n, te_p + te_n

# ------------------------------------------------------------
# 1b) Copy images and create labels IN /kaggle/working ONLY
#     - If input label is missing or empty -> write empty .txt at destination
# ------------------------------------------------------------
def copy_with_labels(split_name, split_list):
    created_empty = 0
    copied_lbl = 0
    for rec in split_list:
        im: Path = rec["img"]
        lb_in: Path|None = rec["lbl_in"]

        dst_img = OUT/f"dataset/images/{split_name}/{im.name}"
        dst_lbl = OUT/f"dataset/labels/{split_name}/{im.stem}.txt"

        shutil.copy2(im, dst_img)
        if lb_in is not None:
            txt = lb_in.read_text().strip()
            if txt == "":
                dst_lbl.write_text("")  # background
                created_empty += 1
            else:
                shutil.copy2(lb_in, dst_lbl)
                copied_lbl += 1
        else:
            dst_lbl.write_text("")      # background
            created_empty += 1
    print(f"[{split_name}] labels: copied {copied_lbl}, empty-created {created_empty}")

for name, lst in [("train",train),("val",val),("test",test)]:
    copy_with_labels(name, lst)

print(f"Train: {len(train)} | Val: {len(val)} | Test: {len(test)}")
print(f"Positives -> Train:{sum(r['has_obj'] for r in train)}, Val:{sum(r['has_obj'] for r in val)}, Test:{sum(r['has_obj'] for r in test)}")

# ------------------------------------------------------------
# 1c) Sanitize YOLO labels in /kaggle/working (clip to [0,1], drop invalid)
# ------------------------------------------------------------
def sanitize_dir(lbl_dir: Path, class_count=2):
    fixed, dropped = 0, 0
    for f in lbl_dir.glob("*.txt"):
        lines_in = f.read_text().strip().splitlines() if f.exists() else []
        lines_out = []
        for ln in lines_in:
            parts = ln.strip().split()
            if len(parts) != 5: 
                dropped += 1; 
                continue
            try:
                cid = int(float(parts[0])); x,y,w,h = map(float, parts[1:5])
            except Exception:
                dropped += 1; 
                continue
            if not (0 <= cid < class_count):
                dropped += 1; 
                continue
            x = min(max(x, 0.0), 1.0); y = min(max(y, 0.0), 1.0)
            w = min(max(w, 0.0), 1.0); h = min(max(h, 0.0), 1.0)
            if w <= 0 or h <= 0:
                dropped += 1; 
                continue
            lines_out.append(f"{cid} {x:.6f} {y:.6f} {w:.6f} {h:.6f}")
            fixed += 1
        f.write_text("\n".join(lines_out))
    return fixed, dropped

for split in ["train","val","test"]:
    fx, dr = sanitize_dir(OUT/f"dataset/labels/{split}", class_count=2)
    print(f"Sanitized {split}: kept {fx} boxes, dropped {dr}")

# ============================================================
# 2) data.yaml
# ============================================================
names = ['dirt','damage']
DATA_YAML = OUT / "data.yaml"
DATA_YAML.write_text(f"""
path: {OUT}/dataset
train: images/train
val: images/val
test: images/test
names:
  0: {names[0]}
  1: {names[1]}
""".strip())

# ============================================================
# 3) (Optional) Hyperparameter evolution
# ============================================================
if RUN_EVOLVE:
    !yolo evolve data="{str(DATA_YAML)}" model=yolov8m.pt imgsz=640 epochs=20 patience=10 batch=16 project="{PROJECT}" name="evolve" generations=25

# ============================================================
# 4) Train YOLOv8-M longer (strong baseline)
# ============================================================
MODEL = "yolov8m.pt"
EPOCHS = 100
IMGSZ = 640

model = YOLO(MODEL)
train_results = model.train(
    data=str(DATA_YAML),
    epochs=EPOCHS,
    imgsz=IMGSZ,
    batch=16,
    device=0 if torch.cuda.is_available() else 'cpu',
    project=PROJECT,
    name="yolo_m_150e",
    seed=SEED,
    patience=30
)
best_ckpt = WORKDIR/PROJECT/"yolo_m_150e/weights/best.pt"

# ============================================================
# 5) Evaluation: standard + TTA, export metrics/plots
# ============================================================
model = YOLO(str(best_ckpt))

val_std = model.val(split='val', imgsz=IMGSZ, save_json=True, plots=True, project=PROJECT, name="val_std")
test_std = model.val(split='test', imgsz=IMGSZ, save_json=True, plots=True, project=PROJECT, name="test_std")
test_tta = model.val(split='test', imgsz=IMGSZ, augment=True, save_json=False, plots=False, project=PROJECT, name="test_tta")

def metrics_to_row(tag, m):
    return {
        "tag": tag,
        "map50_95": float(m.box.map),
        "map50": float(m.box.map50),
        "map75": float(m.box.map75),
        "precision": float(m.box.mp),
        "recall": float(m.box.mr)
    }

rows = [
    metrics_to_row("val_std", val_std),
    metrics_to_row("test_std", test_std),
    metrics_to_row("test_tta", test_tta)
]
pd.DataFrame(rows).to_csv(OUT/"artifacts/summary_metrics.csv", index=False)

# Per-class AP
try:
    per_class_ap = getattr(test_std.box, "maps", None)
    if per_class_ap is not None and len(per_class_ap) == len(names):
        pd.DataFrame({"class": names, "AP50-95": per_class_ap}).to_csv(OUT/"artifacts/per_class_ap.csv", index=False)
except Exception as e:
    print("Per-class AP export skipped:", e)

print("Std test mAP50-95:", float(test_std.box.map), "| TTA:", float(test_tta.box.map))

# Qualitative predictions for figures
pred_dir = OUT / "artifacts/pred_samples"
pred_dir.mkdir(parents=True, exist_ok=True)
_ = model.predict(source=str(OUT/"dataset/images/test"), imgsz=IMGSZ, save=True, conf=0.25, project=str(pred_dir), name="preds")

# ============================================================
# 6) Structured detections + severity scoring
# ============================================================
H, W = 371, 586  # dataset resolution

def detect_image(im_path, conf=0.25):
    res = model.predict(source=str(im_path), imgsz=IMGSZ, conf=conf, verbose=False)
    dets = []
    for r in res:
        if r.boxes is None: 
            continue
        for b in r.boxes:
            cls = int(b.cls.item()); c = float(b.conf.item())
            x1,y1,x2,y2 = b.xyxy.cpu().numpy().tolist()[0]
            dets.append({"cls": names[cls], "conf": c, "bbox_xyxy": [x1,y1,x2,y2]})
    return dets

test_imgs = sorted((OUT/"dataset/images/test").glob("*.*"))
records = []
for im in tqdm(test_imgs, desc="Structuring detections"):
    dets = detect_image(im, conf=0.25)
    severity = 0.0
    for d in dets:
        x1,y1,x2,y2 = d["bbox_xyxy"]
        area = max(0,(x2-x1))*max(0,(y2-y1))/(W*H)
        w_cls = 2.0 if d["cls"]=="damage" else 1.0
        severity += w_cls * area * d["conf"]
    records.append({
        "image": im.name,
        "num_dirt": sum(1 for d in dets if d["cls"]=="dirt"),
        "num_damage": sum(1 for d in dets if d["cls"]=="damage"),
        "severity_score": round(severity, 6),
        "detections_json": json.dumps(dets)
    })
det_df = pd.DataFrame(records)
det_df.to_csv(OUT/"artifacts/detections_test.csv", index=False)

summary = {
    "images_tested": int(len(det_df)),
    "images_with_damage": int((det_df['num_damage']>0).sum()),
    "images_with_dirt_only": int(((det_df['num_damage']==0) & (det_df['num_dirt']>0)).sum()),
    "mean_severity": float(det_df['severity_score'].mean()),
    "median_severity": float(det_df['severity_score'].median()),
    "p95_severity": float(det_df['severity_score'].quantile(0.95))
}
pd.DataFrame([summary]).to_csv(OUT/"artifacts/executive_summary.csv", index=False)
print("Executive summary:", summary)

# ============================================================
# 7) Latency benchmarks (FPS)
# ============================================================
subset = test_imgs[:200] if len(test_imgs) > 200 else test_imgs
t0 = time.time()
for im in subset:
    _ = model.predict(source=str(im), imgsz=IMGSZ, conf=0.25, verbose=False)
t1 = time.time()
fps = len(subset) / (t1 - t0)
pd.DataFrame([{"images": len(subset), "seconds": (t1-t0), "fps": fps, "imgsz": IMGSZ, "model": MODEL}]).to_csv(OUT/"artifacts/inference_fps.csv", index=False)
print(f"Inference FPS on {len(subset)} test images: {fps:.2f}")

# ============================================================
# 8) Optional VLM + LLM inspection reports (with ablation)
# ============================================================
scene_caps = {}
if USE_VLM:
    from transformers import BlipProcessor, BlipForConditionalGeneration
    from PIL import Image
    blip_model = BlipForConditionalGeneration.from_pretrained("Salesforce/blip-image-captioning-base")
    blip_proc  = BlipProcessor.from_pretrained("Salesforce/blip-image-captioning-base")
    blip_model.to("cuda" if torch.cuda.is_available() else "cpu")

    cap_imgs = test_imgs[:400]
    for im in tqdm(cap_imgs, desc="BLIP captions"):
        try:
            img = Image.open(im).convert("RGB")
            inputs = blip_proc(images=img, return_tensors="pt").to(blip_model.device)
            out = blip_model.generate(**inputs, max_new_tokens=30)
            cap = blip_proc.decode(out[0], skip_special_tokens=True)
            scene_caps[im.name] = cap
        except Exception:
            scene_caps[im.name] = ""
    with open(OUT/"artifacts/scene_captions.json","w") as f:
        json.dump(scene_caps, f, indent=2)

def risk_level(s): 
    return "high" if s>0.05 else ("moderate" if s>=0.01 else "low")

reports_with, reports_wo = [], []
if USE_LLM:
    from transformers import AutoTokenizer, AutoModelForSeq2SeqLM
    tok = AutoTokenizer.from_pretrained("google/flan-t5-base")
    llm = AutoModelForSeq2SeqLM.from_pretrained("google/flan-t5-base")
    llm.to("cuda" if torch.cuda.is_available() else "cpu")

    merged = det_df.copy()
    if USE_VLM: merged["scene_caption"] = merged["image"].map(scene_caps).fillna("")

    def gen_report(scene_text, dmg, dirt, sev):
        prompt = f"""You are a UAV inspection engineer. Write a concise professional report (2–4 sentences) for a wind-turbine surface image.

Scene: {scene_text if scene_text else "UAV close-up of turbine blade/surface"}
Detections: damage={dmg}, dirt={dirt}
Severity score (0–1+): {sev:.3f}

Rules:
- If damage>0, recommend a maintenance action.
- State risk level: low (<0.01), moderate (0.01–0.05), high (>0.05).
- No hedging; precise wording; avoid repetition."""
        inputs = tok(prompt, return_tensors="pt").to(llm.device)
        out = llm.generate(**inputs, max_new_tokens=120)
        return tok.decode(out[0], skip_special_tokens=True)

    subset_rows = merged.sample(min(150, len(merged)), random_state=SEED)

    t0 = time.time()
    for _, r in subset_rows.iterrows():
        txt = gen_report(r.get("scene_caption",""), int(r["num_damage"]), int(r["num_dirt"]), float(r["severity_score"]))
        reports_with.append({"image": r["image"], "risk_level": risk_level(float(r["severity_score"])), "inspection_report": txt, "scene_caption": r.get("scene_caption","")})
    t1 = time.time()

    t2 = time.time()
    for _, r in subset_rows.iterrows():
        txt = gen_report("", int(r["num_damage"]), int(r["num_dirt"]), float(r["severity_score"]))
        reports_wo.append({"image": r["image"], "risk_level": risk_level(float(r["severity_score"])), "inspection_report": txt})
    t3 = time.time()

    pd.DataFrame(reports_with).to_csv(OUT/"artifacts/inspection_reports_with_blip.csv", index=False)
    pd.DataFrame(reports_wo).to_csv(OUT/"artifacts/inspection_reports_without_blip.csv", index=False)

    pd.DataFrame([
        {"variant":"with_blip", "samples":len(reports_with), "seconds":(t1-t0), "sec_per_report": (t1-t0)/max(1,len(reports_with))},
        {"variant":"without_blip", "samples":len(reports_wo), "seconds":(t3-t2), "sec_per_report": (t3-t2)/max(1,len(reports_wo))}
    ]).to_csv(OUT/"artifacts/llm_latency.csv", index=False)

# ============================================================
# 9) Citation note
# ============================================================
CITATIONS = """
If you use this dataset, please cite:
Foster, Ashley; Best, Oscar; Gianni, Mario; Khan, Asiya; Collins, Kerry; Sharma, Sanjay (2022),
"Drone Footage Wind Turbine Surface Damage Detection", IEEE IVMSP 2022 (Accepted).
Original dataset: SHIHAVUDDIN, A.S.M.; Chen, X. (2018), “DTU - Drone inspection images of wind turbine”, Mendeley Data, V2, doi:10.17632/hd96prn3nc.2.
"""
with open(OUT/"artifacts/CITATIONS.txt","w") as f:
    f.write(CITATIONS.strip())

print("\n=== Artifacts for your paper ===")
for p in [
    WORKDIR/PROJECT/"yolo_m_150e/weights/best.pt",
    OUT/"artifacts/summary_metrics.csv",
    OUT/"artifacts/per_class_ap.csv",
    WORKDIR/PROJECT/"test_std",     # PR curves & confusion matrix PNGs
    OUT/"artifacts/pred_samples/preds",
    OUT/"artifacts/detections_test.csv",
    OUT/"artifacts/executive_summary.csv",
    OUT/"artifacts/inference_fps.csv",
    OUT/"artifacts/inspection_reports_with_blip.csv",
    OUT/"artifacts/inspection_reports_without_blip.csv",
    OUT/"artifacts/llm_latency.csv",
]:
    print(" -", p)


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.8/41.8 kB 1.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.8/43.8 kB 2.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 872.2/872.2 kB 16.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.1/9.1 MB 94.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 315.1/315.1 kB 16.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 69.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 868.8/868.8 kB 36.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.6/3.6 MB 84.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 5.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 50.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 43.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 27.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664

100%|██████████| 49.7M/49.7M [00:00<00:00, 181MB/s]


New https://pypi.org/project/ultralytics/8.3.178 available 😃 Update with 'pip install -U ultralytics'
Ultralytics YOLOv8.2.84 🚀 Python-3.11.13 torch-2.6.0+cu124 CUDA:0 (Tesla T4, 15095MiB)
engine/trainer: task=detect, mode=train, model=yolov8m.pt, data=/kaggle/working/windturbine_yolo_llm_strong/data.yaml, epochs=100, time=None, patience=30, batch=16, imgsz=640, save=True, save_period=-1, cache=False, device=0, workers=8, project=windturbine_yolo_llm_strong, name=yolo_m_150e, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=42, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=False, agnostic_nms=False, classes=No

100%|██████████| 755k/755k [00:00<00:00, 18.2MB/s]
E0000 00:00:1755088470.152683      19 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1755088470.252256      19 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


Overriding model.yaml nc=80 with nc=2

                   from  n    params  module                                       arguments                     
  0                  -1  1      1392  ultralytics.nn.modules.conv.Conv             [3, 48, 3, 2]                 
  1                  -1  1     41664  ultralytics.nn.modules.conv.Conv             [48, 96, 3, 2]                
  2                  -1  2    111360  ultralytics.nn.modules.block.C2f             [96, 96, 2, True]             
  3                  -1  1    166272  ultralytics.nn.modules.conv.Conv             [96, 192, 3, 2]               
  4                  -1  4    813312  ultralytics.nn.modules.block.C2f             [192, 192, 4, True]           
  5                  -1  1    664320  ultralytics.nn.modules.conv.Conv             [192, 384, 3, 2]              
  6                  -1  4   3248640  ultralytics.nn.modules.block.C2f             [384, 384, 4, True]           
  7                  -1  1   1991808  ultralytics

...

special_tokens_map.json: 0.00B [00:00, ?B/s]

config.json: 0.00B [00:00, ?B/s]

model.safetensors:   0%|          | 0.00/990M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/147 [00:00<?, ?B/s]


=== Artifacts for your paper ===
 - /kaggle/working/windturbine_yolo_llm_strong/yolo_m_150e/weights/best.pt
 - /kaggle/working/windturbine_yolo_llm_strong/artifacts/summary_metrics.csv
 - /kaggle/working/windturbine_yolo_llm_strong/artifacts/per_class_ap.csv
 - /kaggle/working/windturbine_yolo_llm_strong/test_std
 - /kaggle/working/windturbine_yolo_llm_strong/artifacts/pred_samples/preds
 - /kaggle/working/windturbine_yolo_llm_strong/artifacts/detections_test.csv
 - /kaggle/working/windturbine_yolo_llm_strong/artifacts/executive_summary.csv
 - /kaggle/working/windturbine_yolo_llm_strong/artifacts/inference_fps.csv
 - /kaggle/working/windturbine_yolo_llm_strong/artifacts/inspection_reports_with_blip.csv
 - /kaggle/working/windturbine_yolo_llm_strong/artifacts/inspection_reports_without_blip.csv
 - /kaggle/working/windturbine_yolo_llm_strong/artifacts/llm_latency.csv
